# Load in packages

In [1]:
!pip install pandas
!pip install torch torchvision torchaudio
!pip install pandas
!pip install transformers
!pip install scikit-learn
!pip install torch
!pip install tensorflow
!pip install --upgrade pip
!pip install --upgrade numpy
!pip install --upgrade transformers
!pip install --upgrade tensorflow

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 3.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 4.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [2]:
from transformers import pipeline
import tensorflow as tf
import pandas as pd

2024-12-21 19:46:28.087358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734806788.106942   23239 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734806788.113028   23239 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 19:46:28.135599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load in our two subsets

In [4]:
# Only one of the two lines below should be uncommented
#df = pd.read_csv('/work/VNV/Module 1 - Data acqusition and preparation/output/movie_script_original_df.csv') # for the cornell_sub: uncomment for running with the original labels only
df = pd.read_csv('/work/VNV/Module 1 - Data acqusition and preparation/output/movie_script_genderize_df.csv') # for the cornell + genderize sub: uncomment for running with the orignal labels + genderize labels
df = df.rename(columns={'text_of_the_utterance': 'text'})
#df = df.iloc[:50] # uncommen to have smaller test df
print(df)

    lineID characterID movieID character_name  \
0   L1045           u0      m0         BIANCA   
1   L1044           u2      m0        CAMERON   
2    L985           u0      m0         BIANCA   
3    L984           u2      m0        CAMERON   
4    L925           u0      m0         BIANCA   
5    L924           u2      m0        CAMERON   
6    L872           u0      m0         BIANCA   
7    L871           u2      m0        CAMERON   
8    L870           u0      m0         BIANCA   
9    L869           u0      m0         BIANCA   
10   L868           u2      m0        CAMERON   
11   L867           u0      m0         BIANCA   
12   L866           u2      m0        CAMERON   
13   L865           u2      m0        CAMERON   
14   L864           u0      m0         BIANCA   
15   L863           u2      m0        CAMERON   
16   L862           u0      m0         BIANCA   
17   L861           u2      m0        CAMERON   
18   L860           u0      m0         BIANCA   
19   L699           

# Prepare the data for the classifier
We want to make sure that no rows have more than 512 tokens as that is the maximum for our classifier. Any rows with more than 512 tokens will be removed, as we don't want to split up the rows

### Check if there are any rows with more than 512 tokens

In [5]:
# Convert 'text' column to string 
df['text'] = df['text'].astype(str)

# Count how many rows in 'text' contain more than 512 characters
num_long_texts = df['text'].apply(lambda x: len(x) > 512).sum()

print(f'Number of rows with more than 512 characters: {num_long_texts}')

Number of rows with more than 512 characters: 0


### Remove rows with more than 512 tokens

In [6]:
# Length before removing
print(f"No. rows before removing text over 512 characters: {len(df)}")

# Removing rows where text has more than 512 characters
df = df[df['text'].str.len() <= 512]

# Length after removing
print(f"No. rows after removing text over 512 characters: {len(df)}")

No. rows before removing text over 512 characters: 50
No. rows after removing text over 512 characters: 50


# Load in the classifier

In [7]:
# The BERT classifier
classifier = pipeline("text-classification",
                      model='bhadresh-savani/bert-base-uncased-emotion', 
                      return_all_scores=True,
                      framework="pt")

config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
/home/ucloud/.local/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


# Define functions for later use

**A function for getting the output of the classifier in the best way for us**

In [8]:
def get_emotions(text):
    predictions = classifier(text)
    return {pred['label']: pred['score'] for pred in predictions[0]}  # Returns a dictionary of emotions and scores

**A function for extracting the emotion with the highest score**

In [9]:
# extract the emotion with the highest score
def get_top_emotion(emotion_dict):
    return max(emotion_dict, key=emotion_dict.get)

# Apply the classifier to our data

In [10]:
# Ensure the variables in the text column are strings
df['text'] = df['text'].astype(str)

# Make a new column, here the output of emotions probabilities will be stored as dictionaries
df['emotions'] = df['text'].apply(get_emotions)

# Make a new column, here the emotion with the highest probaility will be stored
df['top_emotion'] = df['emotions'].apply(get_top_emotion)

# Save the df as a .csv file
df.to_csv('/work/VNV/Module 3 - Emotion analysis/output/genderize_emotions_dataset.csv', index = False)

# View the start of the df
print(df.head())

   lineID characterID movieID character_name          text gender  movie_year  \
0  L1045           u0      m0         BIANCA  They do not!      f        1999   
1  L1044           u2      m0        CAMERON   They do to!      m        1999   
2   L985           u0      m0         BIANCA    I hope so.      f        1999   
3   L984           u2      m0        CAMERON     She okay?      m        1999   
4   L925           u0      m0         BIANCA     Let's go.      f        1999   

                                            emotions top_emotion  
0  {'sadness': 0.008004407398402691, 'joy': 0.010...       anger  
1  {'sadness': 0.007097005378454924, 'joy': 0.046...       anger  
2  {'sadness': 0.0004969622241333127, 'joy': 0.99...         joy  
3  {'sadness': 0.010399539023637772, 'joy': 0.958...         joy  
4  {'sadness': 0.019666552543640137, 'joy': 0.945...         joy  


# Sanity check of the model 

In [ ]:
# Extract 10 random rows from the data set
random_rows = df.sample(n=10)  
print(random_rows)